### List of what you need to do:
- ~~Define the Queue (generic)~~
- ~~Define the Q-Table (just empty table with all possible states)~~
- Start Q-Learning Loop

In [18]:
# All Imports and Constants
import sys
!{sys.executable} -m pip install gym
import numpy as np
import gym
from gym import spaces
import copy

LEARNING_RATE = 0.1
DISCOUNT = 0.95
EPISODES = 500

# parameters for epsilon decay policy
EPSILON = 1.0 # not a constant, going to be decayed
START_EPSILON_DECAYING = 1
END_EPSILON_DECAYING = EPISODES // 2
epsilon_decay_value = EPSILON / (END_EPSILON_DECAYING - START_EPSILON_DECAYING) # 1 / (250 - 1) = 1 / 249

# for testing
N_TEST_RUNS = 100
TEST_INTERVAL = 50

MAX_TIMESLOTS = 100
MAX_WAIT_STATE = 50 #used as upper limit in q-table

# Generic Queue Simulator
We want to define a generic queue simulator that has its own arrival rates/mean delay requirements so that we can use it for PQ1, PQ2 and Best-Effort, as well as the other queues such as FIFO, RR, etc.

In [19]:
class QueueSimulator(gym.Env):
    def __init__(self, arrival_rates, mean_delay_requirements, queues_finished_timeslots):
        super(QueueSimulator, self).__init__()
        self.arrival_rates = arrival_rates
        self.mean_delay_requirements = mean_delay_requirements
        self.current_timeslot = 1
        self.queues_finished_timeslots = queues_finished_timeslots # Need some reference to this so that I can use it again after reset
        self.queues = copy.deepcopy(queues_finished_timeslots)
        
        # Graph the total wait times for each queue
        self.queues_total_wait_times = [[], [], []]
        
        # Action Space is 3, because we have 3 queues to choose from
        self.action_space = spaces.Discrete(3)
        
        # We know the observation space are the range of possible and observable values. This is wait times,
        # so wait times can be 0 or infinity technically.
        self.observation_space = spaces.Box(low=np.array([0, 0, 0]), high=np.array([np.inf, np.inf, np.inf]), dtype=np.dtype(int))
   
    # Get current wait times for each queue (so how long a queue has been waiting for on a packet to send)
    def calc_state(self):
        calc_state = [0, 0, 0]
        
        for i, queue in enumerate(self.queues):
            queue_total_wait = 0
            for packet in queue:
                if packet <= self.current_timeslot:
                    queue_total_wait += (self.current_timeslot - packet)
            calc_state[i] = queue_total_wait
            
        return calc_state

    # Every step means you calc state_new, reward, done and info
    # Reward should be based on wait time for a packet, as in give HIGHEST reward when 
    #     queue has had to wait for its entire mean_delay_requirement duration
    def step(self, action):
        # First, check how long each queue has been waiting for (this is the initial state)
        current_state = self.calc_state()
        
        # Now calc reward and delete front package from queue[action]
        reward = 0.0
        if current_state[action]/self.mean_delay_requirements[action] >= 1.0:
            reward = 100.0
        else:
            # Not really sure if 10 is necessary here, probs arbitrary
            reward = (10 / self.mean_delay_requirements[action]) * (current_state[action])
            
        # Reward when the best effort queue is chosen should be really high, but not as high as when mean_delay_requirement is met (100 reward)
        if action == 2:
            reward = 90
        
        # Everytime you transmit a packet, keep track of how long that packet had to wait in queues_total_wait_times to graph later
        if (len(self.queues[action]) > 0):
            self.queues_total_wait_times[action].append(current_state[action])
            del self.queues[action][0]
        
        # Now get new state to send back
        new_state = self.calc_state()
        
        # done = True only when all queues have no packets left
        done = False
        if all(len(queue) == 0 for queue in self.queues):
            done = True

        self.current_timeslot += 1
        return new_state, reward, done, {}
        
    # Need to reset the vars between episodes like timeslots
    # Since q_learning expects state reset too, return the calc_state method
    # Need to add packets back into queue because i deleted them every run lol
    def reset(self):
        self.current_timeslot = 0
        self.queues = copy.deepcopy(self.queues_finished_timeslots)
        return self.calc_state()
        
    def render(self):
        return self


# Pre-populating Queues
If given the arrival_rates and mean_delay_requirements, you could calculate what timeslots a packet will arrive for any number of timeslots. Hence, we believe that you should just 'pre-populate' your queues with the times that packets arrive at, since this simplifies all of the packet arrival/transmission, and you can later use this to measure wait times and give this information to your model to determine an action to take every step of the way.

In [20]:
# Create the basic env and put logic for the actions
arrival_rates = [0.3, 0.25, 0.4]
mean_delay_requirements = [6, 4, np.inf]
# Keep track of current packets by using another array, which has index corresponding to the arrival_rates
queues_packet_status = [0, 0, 0]
# See all timeslots where a queue finished transmitting a packet
queues_finished_timeslots = [[], [], []]

# At each time interval, increment each queue's current packet status by the arrival rate amount
# if packet status >= 1, get the extra amt above 1 and change packet status to just that
for timeslot in range(1, MAX_TIMESLOTS+1):
    for current_queue in range (len(arrival_rates)):
        queues_packet_status[current_queue] += arrival_rates[current_queue]
        
        if queues_packet_status[current_queue] >= 1.0:
            queues_finished_timeslots[current_queue].append(timeslot)
            queues_packet_status[current_queue] -= 1.0

# Result of queues_finished_timeslots
print('{0} -> Length = {1}'.format(queues_finished_timeslots[0], len(queues_finished_timeslots[0])))
print('{0} -> Length = {1}'.format(queues_finished_timeslots[1], len(queues_finished_timeslots[1])))
print('{0} -> Length = {1}'.format(queues_finished_timeslots[2], len(queues_finished_timeslots[2])))


[4, 7, 11, 14, 17, 21, 24, 27, 31, 34, 37, 41, 44, 47, 51, 54, 57, 61, 64, 67, 71, 74, 77, 81, 84, 87, 91, 94, 97] -> Length = 29
[4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52, 56, 60, 64, 68, 72, 76, 80, 84, 88, 92, 96, 100] -> Length = 25
[3, 5, 8, 10, 13, 15, 18, 20, 23, 25, 28, 30, 33, 35, 38, 40, 43, 45, 48, 50, 53, 55, 58, 60, 63, 65, 68, 70, 73, 75, 78, 80, 83, 85, 88, 90, 93, 95, 98, 100] -> Length = 40


'queues_finished_timeslots' is essentially our environment. We want to take this environment and apply it to a Q-Table, where the Q-Table represents all possible states (states being how long a queue has waited), then some reward for an action given a specific state. Below is what the Q-Table could look like, with a state e.g. (1, 0, 3) saying that the first packet in queue 1 has waited 1 timeslots, queue 2 has waited 0 timeslots, and queue 3 has waited 3 timeslots.

|                       | 0           | 1           | 2           |
|-----------------------|:-----------:|-------------|-------------|
| State (0, 0, 0)       | some-reward | some-reward | some-reward |
| State (0, 0, 1)       | some-reward | some-reward | some-reward |
| State (0, 1, 1)       | "         " | "         " | "         " |
| State (  ...  )       |             |             |             |
| State (Inf, Inf, Inf) | some-reward | some-reward | some-reward |

# Setting Up Q-Learning
First, we need to make a Q-Table with empty values. Access to an entry is given with the state (x, y, z).
Also, the upper state cannot be infinity so we can choose an arbitrary value (but still one that is somewhat realistic).

In [21]:
def create_q_table():
    q_table = {}
    for q1 in range (MAX_WAIT_STATE+1):
        for q2 in range (MAX_WAIT_STATE+1):
            for q3 in range (MAX_WAIT_STATE+1):
                q_table[q1, q2, q3] = np.zeros(3)
    return q_table

# Start Q-Learning Loop
1. For every episode, do another 'until done' loop
2. While not done:
    - Get a random chance, and either get value from Q-Table (exploit) or do random action (explore)
    - Do next step() for env
    - Update Q-Table and any other variables
3. Update epsilon
4. Can do some update per episode, but if we're doing minimum 500 episodes more likely to do some performance check every X interval e.g. every 50 episodes get some check in.

In [22]:
def max_limit_state(state):
    for i, wait_time in enumerate(state):
        if wait_time > MAX_WAIT_STATE:
            state[i] = MAX_WAIT_STATE
    return state

def q_learning(env, q_table):
    for episode in range(EPISODES):
        print('New Episode')
        # Reset all variables per episode
        done = False
        state = env.reset()
        epsilon = EPSILON
        steps = 0

        # Either do action from QTable or random action
        while not done:
            if np.random.random() < 1 - epsilon:
                action = np.argmax(q_table[state])
            else:
                action = env.action_space.sample()
                
            # Get the next state, reward, new done value, and info (not sure what this is)
            # Also, in early episodes the queues can wait for very long times. Put a limit on the wait times in new_state
            new_state, reward, done, info = env.step(action)
            new_state = max_limit_state(new_state)
            print('State: {0}'.format(new_state))

            # Update QTable and calc reward. Not sure why current_q in example is 'discretState+(action,)'
            # Note: Need to convert new_state to tuple since q_table entries are as tuples
            q_table_key = tuple(new_state)
            new_max_q = np.max(q_table[q_table_key])
            current_q = q_table[q_table_key][action]
            q_table[q_table_key][action] = (1 - LEARNING_RATE)*current_q + LEARNING_RATE*(reward + DISCOUNT*new_max_q)

            state = new_state
            steps += 1

        # Finished done loop, update epsilon
        if END_EPSILON_DECAYING >= episode and episode >= START_EPSILON_DECAYING:
            epsilon -= epsilon_decay_value

        # Print progress every X episodes
        if episode % TEST_INTERVAL == 0 and episode != 0:
            print('Finished {0} episodes'.format(episode))

    print('Finished all episodes')
    env.close()
    return q_table

env = QueueSimulator(arrival_rates, mean_delay_requirements, queues_finished_timeslots)
q_table = create_q_table()
result_q_table = q_learning(env, q_table)

New Episode
State: [0, 0, 0]
State: [0, 0, 0]
State: [0, 0, 0]
State: [0, 0, 0]
State: [0, 0, 0]
State: [0, 0, 0]
State: [0, 0, 0]
State: [0, 0, 0]
State: [0, 0, 0]
State: [0, 1, 0]
State: [0, 2, 0]
State: [0, 3, 0]
State: [0, 4, 0]
State: [0, 6, 0]
State: [0, 2, 0]
State: [0, 3, 0]
State: [0, 0, 0]
State: [0, 1, 0]
State: [0, 2, 0]
State: [0, 3, 0]
State: [0, 4, 0]
State: [0, 6, 0]
State: [0, 2, 0]
State: [0, 0, 0]
State: [0, 0, 0]
State: [0, 1, 0]
State: [0, 0, 0]
State: [0, 0, 0]
State: [0, 0, 0]
State: [0, 0, 0]
State: [0, 0, 0]
State: [0, 0, 0]
State: [0, 0, 0]
State: [0, 0, 0]
State: [0, 0, 0]
State: [0, 0, 0]
State: [0, 0, 1]
State: [0, 0, 2]
State: [0, 0, 3]
State: [0, 0, 5]
State: [0, 0, 7]
State: [0, 0, 10]
State: [0, 0, 13]
State: [0, 0, 16]
State: [0, 0, 20]
State: [0, 0, 24]
State: [0, 0, 29]
State: [0, 0, 34]
State: [0, 0, 26]
State: [0, 0, 31]
State: [0, 0, 24]
State: [0, 0, 29]
State: [0, 0, 34]
State: [0, 0, 39]
State: [0, 0, 45]
State: [0, 0, 50]
State: [0, 0, 50]
Sta

In [23]:
print(result_q_table[(0, 0, 1)])

[1608.72650576 1610.24565959 1699.68473734]
